## 프로젝트: 멋진 작사가 만들기

### Library Import

In [1]:
import os, re, glob
import numpy as np
import tensorflow as tf

### Step 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


### Step 3. 데이터 정제

가사에 포함된 Verse, Chorus와 같은 요소들은 가사 내용이 아니므로 제외하고자 한다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장
     #sentence에 [Verse 1]과 같이 []로만 구성되어 있는 문장을 삭제한다. ([] 데이터 제거.1)
    if re.match(r'\[[^)]*\]', sentence): continue

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 1
    # 문장에 가사와 함께 포함된 [Chorus]와 같은 요소들을 제거한다. ([] 데이터 제거.2)
    sentence = re.sub(r'\[[^)]*\]', '', sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if re.match(r'\[[^)]*\]', sentence): continue
    if len(sentence.split(" ")) >= 12 : continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]
print(len(corpus))

150911


In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=14)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
corpus[:10]

[[   2   43   64 ...    0    0    0]
 [   2    9 2958 ...    0    0    0]
 [   2   43   64 ...    0    0    0]
 ...
 [   2  551   20 ...    0    0    0]
 [   2  121   36 ...    3    0    0]
 [   2    5   23 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fb0b0a349d0>


['<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>',
 '<start> before it gets too late <end>',
 '<start> it s up to me and you <end>']

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  43  64  73 700   3   0   0   0   0   0   0   0]
[ 43  64  73 700   3   0   0   0   0   0   0   0   0]


### Step 4. 평가 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (120728, 13)
Target Train: (120728, 13)


### Step 5. 인공지능 만들기

In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 13, 12001), dtype=float32, numpy=
array([[[-8.94301193e-05, -6.29786655e-06, -5.16679611e-05, ...,
         -6.24375461e-05,  9.96769595e-05, -9.46724685e-05],
        [ 8.74621037e-07, -2.85960530e-04, -3.98364151e-04, ...,
         -1.66154859e-04,  1.53619927e-04,  7.93450454e-05],
        [-1.43335114e-04, -3.78931494e-04, -6.04001980e-04, ...,
         -2.64297938e-04,  7.82661227e-05,  3.52073112e-05],
        ...,
        [ 8.77041603e-05, -1.34525006e-03,  2.04397402e-05, ...,
         -1.23841874e-03,  5.31490019e-04, -1.35256210e-03],
        [ 2.44222872e-04, -1.68232038e-03,  2.13212352e-05, ...,
         -1.73617085e-03,  6.21623185e-04, -1.71871926e-03],
        [ 4.28505824e-04, -1.99870998e-03, -1.97370483e-07, ...,
         -2.19845911e-03,  7.34861300e-04, -2.02441285e-03]],

       [[-8.94301193e-05, -6.29786655e-06, -5.16679611e-05, ...,
         -6.24375461e-05,  9.96769595e-05, -9.46724685e-05],
        [-5.15790191e-04, -1.60296840e-05, -2

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=15)

Epoch 1/15
589/589 [==============================] - 174s 291ms/step - loss: 4.1288
Epoch 2/15
589/589 [==============================] - 173s 293ms/step - loss: 3.1841
Epoch 3/15
589/589 [==============================] - 173s 293ms/step - loss: 2.9882
Epoch 4/15
589/589 [==============================] - 173s 293ms/step - loss: 2.8284
Epoch 5/15
589/589 [==============================] - 172s 292ms/step - loss: 2.7038
Epoch 6/15
589/589 [==============================] - 172s 292ms/step - loss: 2.5934
Epoch 7/15
589/589 [==============================] - 172s 292ms/step - loss: 2.4901
Epoch 8/15
589/589 [==============================] - 171s 291ms/step - loss: 2.3909
Epoch 9/15
589/589 [==============================] - 172s 291ms/step - loss: 2.2958
Epoch 10/15
589/589 [==============================] - 171s 290ms/step - loss: 2.2158
Epoch 11/15
589/589 [==============================] - 172s 291ms/step - loss: 2.1329
Epoch 12/15
589/589 [==============================] - 171s 290

텍스트 생성모델의 validation loss를 2.2 이하로 줄이기 위해 Epoch를 조정해준다.

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

### Review

문제에서 권장하는 데이터 개수인 124,960개의 데이터 이하로 줄이기 위해, token의 개수를 15개 이하로 줄이는 것 외에 시도해 본 것
- sentence의 수집 과정에서 단어의 수를 제한했다.
- [Verse]와 같은 Sentence 자체가 [ ]로 이루어진 요소를 제거했다. ([ ] 데이터 제거.1)
- [Chorus]와 같이 Sentence에 포함된 [ ]로 이루어진 요소를 제거했다. ([ ] 데이터 제거.2)

시도해 보려고 했지만 실패한 것
- 가사 내부에서 같은 Sentence가 여러 번 반복된 경우를 ndarray의 행에서 삭제하고자 시도해 보았지만, 아직 numpy 배열에 익숙하지 않아서인지 성공하지 못했다.

: sentence를 이루는 단어의 수를 제한하는 것보다는 불필요한 요소를 줄이는 방향으로 데이터의 양을  줄이고자 하였지만, 생각보다 데이터의 수를 많이 줄이지 못해 아쉬웠고, 내가 어떤 부분에서 부족한지를 좀 더 알게 되어 흥미롭게 프로젝트를 진행할 수 있었다.